In [1]:
%%html
<style>
.prompt_container { display: none !important; }
.prompt { display: none !important; }
.run_this_cell { display: none !important; }

.slides {
    position: absolute;
    top: 0;
    left: 0;
}
</style>

# ProgRes, Part IIb

# Web servers

Fabien Mathieu - fabien.mathieu@normalesup.org

Sébastien Tixeuil - Sebastien.Tixeuil@lip6.fr

# Methodology

- Course, practicals, and mini-projects are made on notebooks (jupyter or jupyterlab)
- Available on Github: https://github.com/balouf/progres
- This means you will send your practical notebooks. 
- Please put your name(s) **on the filename AND inside** as well!

# The rules

https://github.com/balouf/progres/blob/main/rules.ipynb

# Roadmap

- Part I: done
- Part II: done
- Part II: Web services
  - Last week: Client side
  - This week: Server side
    - Server module
    - Testing
    - Web Server + API


## Create an API in Python

# Reminder

HTTP can be used to communicate:
- Between the server and the user (frontend)
- Internally, between the server and other services (backend)

API (Application Programming Interface) is a set of exposed methods for interaction between programms.

REST is a *code of conduct* for HTTP-based API.

# Anatomy of a Web server

To process a HTTP request, you need three elements:
- Receive the request
- Dispatch the request
- Process the request

Some programs are all-in-one, other are dedicated.

# Receive (the actual server)

- Apache
- Nginx
- lighttpd

# Send to process (the dispatcher)

- Can depend on the domain name (reverse proxy)
- Can be another, internal, HTTP request
- Examples: Nginx, HAProxy

# Dispatch example

- A general nginx server listen on 80, the only open port of a server (frontend)
- https://demo1.mydomain.com:80 goes to internal nginx backend on port 8080
- https://demo2.mydomain.com:80 goes to internal Apache backend on port 8090
- The three can be on the same or different machines

# Process (the application)

- File serving
- Server-side computation (e.g. PHP)
- Internal micro-services requests (REST API)
- Examples: Apache, **Python frameworks**

# Web Server GateWay Interface (WSGI)

It is a standard (like REST API) for Python Web apps/services

- Goal: specify HTTP <-> python conversion
- Benefit: can setup HTTP and Python architectures independently
- Most Python frameworks are already WSGI-ready
- Some dedicated WSGI middlewares: uWSGI, Gunicorn, Waitress
- You (developer) don't need to know how it works

# Web application frameworks (Python)

Many libraries / framework exist in Python.

From https://www.educative.io/blog/top-python-web-frameworks

1. Flask: light, yet powerful. Adapted to middle-complexity websites. This year proposal!
2. Django: ultra-powerful, ultra-complete. Adapted to production-ready complex websites.
3. FastAPI: light, robust, easy-to-learn, production-ready.
4. CherryPy: Flask simpler and more pythonic. Adapted to professors.
5. Bottle: very light, standalone. Adapted to former courses.


# Web application frameworks (Python)

- Thanks to WSGI and similar efforts, all frameworks are 80% similar
  - If you know Flask, you can read Bottle code.
  - Small efforts required for conversion.
- Some frameworks provide the full stack, server included
  - Some are just for small tests, e.g. Bottle server.
  - Some are production-ready.
  - For scalable production-ready, use a dedicated server (nginx).

# The Flask framework

- **Routing**: Requests to function-call mapping with support for clean and dynamic URLs.
- **Templates**: Use your own templates or Jinja (i.e. Jinja2) templates.
- **Utilities**: Convenient access to form data, file uploads, cookies, headers and other HTTP-related metadata.
- **Server**: OK for tests, development, and courses.

# Hello world example

In [3]:
from flask import Flask

app = Flask("MyApp")

@app.route('/hello/<name>')
def index(name):
    return 'Hello ' + name + '!'

app.run(host='localhost', 
        port=8080, debug=True, 
        use_reloader=False)

 * Serving Flask app 'MyApp'
 * Debug mode: on


 * Running on http://localhost:8080
Press CTRL+C to quit
127.0.0.1 - - [05/Dec/2025 12:11:51] "GET /hello/World HTTP/1.1" 200 -
127.0.0.1 - - [05/Dec/2025 12:11:51] "GET /favicon.ico HTTP/1.1" 404 -


Check that the server works: http://localhost:8080/hello/World

# Hello world example explained

- `route` is a Python decorator.
  - Decorators are used to modify the behavior of a method
  - Here `@route` links the method `index(name)` to the route `/hello/<name>`
- `run` starts the Web server
  - ``host=`localhost` ``: listen local incoming connections
  - `port`: listening port
  - `debug`: enables `auto-reload`
  - `use_reloader`: a trick for using Flask inside Jupyter

# Intermezzo: Jupyter/IPython magics 

- IPython Magics are powerful commands that change the behavior of a code cell.
- Here we will use some to avoid switching back and forth between the notebook and command line.
- Using magics is never mandatory but often helpful
- If you want to learn more about magics: https://ipython.readthedocs.io/en/stable/interactive/magics.html

# Hello world example with magics

In [6]:
%%writefile run.py
from flask import Flask
app = Flask("MyApp")
@app.route('/hello/<name>')
def index(name):
    return f"Hello {name}!"
app.run(debug=True)

Overwriting run.py


In [11]:
!wt uv run run.py

Check that the server works: http://localhost:5000/hello/Students

Note: `!wt uv run...` is for my config (Windows+UV). Linux/Max: `!python ...` should be enough.

# Hello world example a bit cleaner

In [12]:
%%writefile app.py
from flask import Flask
app = Flask("MyApp")
@app.route('/')
def alive():
    routes = "\n".join(f"<li><a href='{rule.rule}'>{rule.rule}</a></li>" 
              for rule in app.url_map.iter_rules())
    return f"Server running. Available routes:<br><ul>{routes}</ul>"

Overwriting app.py


In [13]:
%%writefile run.py
from app import app
@app.route('/hello/<name>')
def index(name):
    return f'Hello {name}!'
app.run(debug=True)

Overwriting run.py


http://localhost:5000

# Check result with requests

In [15]:
from IPython.display import display, HTML
from requests import get, post, Session
s = Session()
base_url = "http://127.0.0.1:5000"
def check(url, method=None, verbose=False, show=False):
    if method is None:
        method = s.get
    r = method(base_url+url)
    if show:
        display(HTML(r.text))
        return None
    print(f"Content: {r.text}")
    if verbose:
        print(f"Status: {r.status_code}")
        print(f"Headers: {r.headers}")

# Filters: int, float,

In [16]:
%%writefile run.py
from app import app
@app.route('/square/<string:i>')
def square_string(i):
    return f'The square of {i} is {i} X {i}.'
@app.route('/square/<float:i>')
def square_float(i):
    return f'The (rounded) square of {i} is {int(i**2)}.'
@app.route('/square/<int:i>')
def square_int(i):
    return f'The square of {i} is {i**2}.'
app.run(debug=True)

Overwriting run.py


In [17]:
check('/square/Two')
check('/square/42')
check('/square/3.14159')

Content: The square of Two is Two X Two.
Content: The square of 42 is 1764.
Content: The (rounded) square of 3.14159 is 9.


# Manual filter

In [18]:
%%writefile run.py
from app import app
import re
@app.route('/slug/<id_slug>')
def slug(id_slug):
    try:
        sid, name = re.fullmatch(r'([0-9]+)-(.*)', id_slug).groups()
        return f'Slug {name} has id {sid}.'
    except AttributeError:
        return "URL does not exist", 404
app.run(debug=True)

Overwriting run.py


In [19]:
check("/slug/123-for-five-six")
check("/slug/bad-format", verbose=True)

Content: Slug for-five-six has id 123.
Content: URL does not exist
Status: 404
Headers: {'Server': 'Werkzeug/3.1.4 Python/3.13.7', 'Date': 'Fri, 05 Dec 2025 11:43:58 GMT', 'Content-Type': 'text/html; charset=utf-8', 'Content-Length': '18', 'Connection': 'close'}


# Serving files

In [20]:
%%writefile run.py
from app import app
from flask import send_file
@app.route('/open/<path:filepath>')
def open_file(filepath):
    return send_file(filepath)
app.run(debug=True)

Overwriting run.py


In [21]:
%%writefile hello.html
<h1>HELLO WORlD!</h1>

Overwriting hello.html


In [22]:
check("/open/hello.html", show=True)

# Debug mode: other side effect

In [23]:
check("/open/bye.html", show=True)

# Specify the method (GET, POST, ...)

In [24]:
%%writefile run.py
from flask import request
from app import app
@app.route('/square/<int:i>', methods=['post'])  # @app.post works too
def square(i):
    return f'The square of {i} is {i**2}, method is {request.method}.'
app.run(debug=True)

Overwriting run.py


In [25]:
check("/square/42", verbose=True)

Content: <!doctype html>
<html lang=en>
<title>405 Method Not Allowed</title>
<h1>Method Not Allowed</h1>
<p>The method is not allowed for the requested URL.</p>

Status: 405
Headers: {'Server': 'Werkzeug/3.1.4 Python/3.13.7', 'Date': 'Fri, 05 Dec 2025 11:44:28 GMT', 'Content-Type': 'text/html; charset=utf-8', 'Allow': 'OPTIONS, POST', 'Content-Length': '153', 'Connection': 'close'}


# A route can return...

- Content (text, json, ...)
- A tuple content, status (=200), headers (=dict with default values)
- A Response object (packs content, status, and headers, in a nice way)

# Send text and headers

In [26]:
%%writefile run.py
from app import app

@app.get('/content')
def content():
    return "Content with header", {"Server": "Custom client", "a": 42, "b": None}
app.run(debug=True)

Overwriting run.py


In [27]:
check("/content", verbose=True)

Content: Content with header
Status: 200
Headers: {'Server': 'Werkzeug/3.1.4 Python/3.13.7, Custom client', 'Date': 'Fri, 05 Dec 2025 11:45:01 GMT', 'a': '42', 'b': 'None', 'Content-Type': 'text/html; charset=utf-8', 'Content-Length': '19', 'Connection': 'close'}


# Send file and status

In [28]:
%%writefile run.py
from app import app
from flask import send_file

@app.get('/content')
def content():
    return send_file("run.py"), 302
app.run(debug=True)

Overwriting run.py


In [29]:
check("/content", verbose=True)

Content: from app import app
from flask import send_file

@app.get('/content')
def content():
    return send_file("run.py"), 302
app.run(debug=True)

Status: 302
Headers: {'Server': 'Werkzeug/3.1.4 Python/3.13.7', 'Date': 'Fri, 05 Dec 2025 11:45:07 GMT, Fri, 05 Dec 2025 11:45:07 GMT', 'Content-Disposition': 'inline; filename=run.py', 'Content-Type': 'text/x-python; charset=utf-8', 'Content-Length': '148', 'Last-Modified': 'Fri, 05 Dec 2025 11:45:03 GMT', 'Cache-Control': 'no-cache', 'ETag': '"1764935103.1981-148-2886080044"', 'Connection': 'close'}


# Send json

In [30]:
%%writefile run.py
from app import app

@app.get('/content')
def content():
    return {'a': 42, 'b': None, 'c': "Été"}
app.run(debug=True)

Overwriting run.py


In [31]:
check("/content", verbose=True)

Content: {
  "a": 42,
  "b": null,
  "c": "\u00c9t\u00e9"
}

Status: 200
Headers: {'Server': 'Werkzeug/3.1.4 Python/3.13.7', 'Date': 'Fri, 05 Dec 2025 11:45:15 GMT', 'Content-Type': 'application/json', 'Content-Length': '51', 'Connection': 'close'}


# Send json

In [32]:
%%writefile run.py
from app import app
from flask import jsonify

@app.get('/content')
def content():
    return jsonify(a=42, b=None, c="Été")
app.run(debug=True)

Overwriting run.py


In [33]:
check("/content", verbose=True)

Content: {
  "a": 42,
  "b": null,
  "c": "\u00c9t\u00e9"
}

Status: 200
Headers: {'Server': 'Werkzeug/3.1.4 Python/3.13.7', 'Date': 'Fri, 05 Dec 2025 11:45:21 GMT', 'Content-Type': 'application/json', 'Content-Length': '51', 'Connection': 'close'}


# Using a Response object

In [34]:
%%writefile run.py
from app import app
from flask import make_response

@app.get('/content')
def content():
    r = make_response("Hello")
    r.status_code = 302
    return r
app.run(debug=True)

Overwriting run.py


In [35]:
check("/content", verbose=True)

Content: Hello
Status: 302
Headers: {'Server': 'Werkzeug/3.1.4 Python/3.13.7', 'Date': 'Fri, 05 Dec 2025 11:45:25 GMT', 'Content-Type': 'text/html; charset=utf-8', 'Content-Length': '5', 'Connection': 'close'}


# Content-Type

In [36]:
%%writefile run.py
from app import app
import json
@app.route('/powers/<int:i>')
def powers_json(i):
    return {'i':i, 'square': i**2, 'cube': i**3}
@app.route('/powers_txt/<int:i>')
def powers_txt(i):
    return json.dumps({'i':i, 'square': i**2, 'cube': i**3})
app.run(debug=True)

Overwriting run.py


http://localhost:5000

- When returning a string, it is encoded with respect to the given content-type (only bytes are sent back to the client)
- Here, the server sends a byte array encoded in utf-8, that represents an object formatted in JSON.

# Sending Errors, Redirects

In [37]:
%%writefile run.py
from app import app
from flask import abort, redirect, send_file
@app.route('/private/<int:id>')
def private(id):
    abort(401, "Sorry, access denied.")
@app.route('/open/<path:filepath>')
def open_file(filepath):
    return send_file(filepath)
@app.route('/json/<stem>')
def getjson(stem):
    return redirect(f"/open/{stem}.json")
app.run(debug=True)

Overwriting run.py


http://localhost:5000

## Testing your code

# Why testing is important?

- Domino error
  - A function `two` returns 3 instead of 2
  - In a far, far away submodule, lies `4**two()`
- Coverage:
  - Are you sure you don't have dead code lying?
  - https://balouf.github.io/stochastic_matching/readme.html
- Quick reaction:
  - When you break things
  - When another dev breaks things
  - When the new release of an obscure package breaks things 

# How to test?

- unittest
  - The original testing framework
  - You should not use it (too heavy!)
- pytest
  - like unittest but simpler
  - You should use it!
- doctests (not covered here but nice!)
  
 https://www.lincs.fr/events/testing-in-python/

# pytest syntax

If it starts with `test`, it's a test.

In [38]:
!del test_*.py

In [39]:
%%writefile test_file.py

def square(x):
    return x**2

def test_square():
    assert square(2)==4

Writing test_file.py


In [41]:
!pytest

============================= test session starts =============================
platform win32 -- Python 3.13.7, pytest-9.0.1, pluggy-1.6.0
rootdir: C:\Users\fabienma\CloudStation\Projets\2022 - Progres
configfile: pyproject.toml
plugins: anyio-3.7.1
collected 1 item

test_file.py .                                                           [100%]

============================== 1 passed in 0.40s ==============================


# Example: arithmetic testing

In [42]:
%%writefile run_maths.py
from app import app

@app.route("/add/<int:i>/<int:j>")
def add(i,j):
    return [i+j]
@app.route("/sub/<int:i>/<int:j>")
def sub(i,j):
    return [i-j]

@app.route("/mul/<int:i>/<int:j>")
def mul(i,j):
    return [i*j]
@app.route("/div/<int:i>/<int:j>")
def div(i,j):
    return [i//j,i%j]
app.run(port=5070, debug=True)

Overwriting run_maths.py


In [44]:
!wt uv run run_maths.py

http://localhost:5070/add/3/6
http://localhost:5070/mul/3/6
http://localhost:5070/sub/3/6
http://localhost:5070/div/32/6

# Example: arithmetic testing

In [45]:
server_ip = "127.0.0.1"
server_port = 5070
r1 = get(f"http://{server_ip}:{server_port}/add/4/5")
print(r1.text)
r2 = get(f"http://{server_ip}:{server_port}/add/7/14")
print(r2.text)

[
  9
]

[
  21
]



# Example: arithmetic testing

In [46]:
%%writefile test_maths.py
from requests import get
from json import loads
server_ip = "127.0.0.1"
server_port = 5070
def test_add():
    r1 = get(f"http://{server_ip}:{server_port}/add/4/5")
    assert loads(r1.text) == [ 9 ]
    r2 = get(f"http://{server_ip}:{server_port}/add/7/14")
    assert loads(r2.text) == [21]

Writing test_maths.py


In [47]:
!pytest

============================= test session starts =============================
platform win32 -- Python 3.13.7, pytest-9.0.1, pluggy-1.6.0
rootdir: C:\Users\fabienma\CloudStation\Projets\2022 - Progres
configfile: pyproject.toml
plugins: anyio-3.7.1
collected 2 items

test_file.py .                                                           [ 50%]
test_maths.py .                                                          [100%]

============================== 2 passed in 0.59s ==============================


# Example: arithmetic testing

In [48]:
%%writefile test_maths.py
from requests import get
from json import loads
server_ip = "127.0.0.1"
server_port = 5070
def test_add():
    r1 = get(f"http://{server_ip}:{server_port}/add/4/5")
    assert loads(r1.text) == [ 9 ]
    r2 = get(f"http://{server_ip}:{server_port}/add/7/14")
    assert loads(r2.text) == [21]
def test_sub():
    r1 = get(f"http://{server_ip}:{server_port}/sub/4/5")
    assert loads(r1.text) == [ 1 ]
    r2 = get(f"http://{server_ip}:{server_port}/sub/2/2")
    assert loads(r2.text) == [0]

Overwriting test_maths.py


In [49]:
!pytest

============================= test session starts =============================
platform win32 -- Python 3.13.7, pytest-9.0.1, pluggy-1.6.0
rootdir: C:\Users\fabienma\CloudStation\Projets\2022 - Progres
configfile: pyproject.toml
plugins: anyio-3.7.1
collected 3 items

test_file.py .                                                           [ 33%]
test_maths.py .F                                                         [100%]

================================== FAILURES ===================================
__________________________________ test_sub ___________________________________

    def test_sub():
        r1 = get(f"http://{server_ip}:{server_port}/sub/4/5")
>       assert loads(r1.text) == [ 1 ]
E       assert [-1] == [1]
E         
E         At index 0 diff: -1 != 1
E         Use -v to get more diff

test_maths.py:12: AssertionError
=========================== short test summary info ===========================
FAILED test_maths.py::test_sub - assert [-1] == [1]
==============

# Example: strings

In [50]:
%%writefile run_string.py
from app import app
@app.route("/to_upper/<s>")
def to_upper(s):
    return s.upper()
app.run(port=5080, debug=True)

Overwriting run_string.py


In [51]:
!wt uv run run_string.py

http://localhost:5080/to_upper/hello

# Example: strings

In [52]:
%%writefile test_strings.py
from requests import get
server_ip = "127.0.0.1"
server_port = 5080
def test_upper():
    r1 = get(f"http://{server_ip}:{server_port}/to_upper/LaTeX")
    assert r1.text == 'LATEX'
    r2 = get(f"http://{server_ip}:{server_port}/to_upper/Été")
    assert r2.text == 'ÉTÉ'

Writing test_strings.py


In [53]:
!pytest

============================= test session starts =============================
platform win32 -- Python 3.13.7, pytest-9.0.1, pluggy-1.6.0
rootdir: C:\Users\fabienma\CloudStation\Projets\2022 - Progres
configfile: pyproject.toml
plugins: anyio-3.7.1
collected 4 items

test_file.py .                                                           [ 25%]
test_maths.py .F                                                         [ 75%]
test_strings.py .                                                        [100%]

================================== FAILURES ===================================
__________________________________ test_sub ___________________________________

    def test_sub():
        r1 = get(f"http://{server_ip}:{server_port}/sub/4/5")
>       assert loads(r1.text) == [ 1 ]
E       assert [-1] == [1]
E         
E         At index 0 diff: -1 != 1
E         Use -v to get more diff

test_maths.py:12: AssertionError
=========================== short test summary info ==============

## API Hour


# A First micro-service example

In [54]:
%%writefile run.py
ip="127.0.0.1"
math_port=5070
from flask import request, Flask
from requests import get
from json import loads
app = Flask("Main")
form = """<form action="/" method="post">
 i: <input name="i" type="text" />
 j: <input name="j" type="text" />
<input value="Add" type="submit" />
</form>"""
@app.get("/")
def input_form():
    return form
@app.post("/")
def process_form():
    i = request.form['i']
    j = request.form['j']
    r = get(f"http://{ip}:{math_port}/add/{i}/{j}")
    l = loads(r.text)
    return f"<h1>{i}+{j}={l[0]}</h1>"    
app.run(debug=True)

Overwriting run.py


http://localhost:5000

# What happened?

- Client (browser) requests a get method to 5000
- 5000 responds to get with a html form to client
- Client requests a post method to 5000
- 5000 requests a get method to 5070
- 5070 responds to get with a json to 5000
- 5000 responds to post with a html result to client

# Another one?

In [55]:
%%writefile run.py
ip="127.0.0.1"
string_port=5080
from flask import Flask, request
app = Flask("Main")
from requests import get
form = """<form action="/" method="post">
 Word: <input name="s" type="text" />
 <input value="To Uppercase" type="submit" />
 </form>"""
@app.get("/")
def input_form():
    return form
@app.post("/")
def process_form():
    s = request.form['s'] # unicode is universal?
    r = get(f"http://{ip}:{string_port}/to_upper/{s}")
    l = r.text
    return f"<h1>{s}.upper()={l}</h1>"    
app.run(debug=True)

Overwriting run.py


http://localhost:5000

# Getting parameters:

- from the URL, e.g. `/entry?order=name`: `request.args['order']`
- from the post data, e.g. a posted form: `request.form['order']`
- from URL or post data: `request.values['order']`
- Retrieve a posted file:
  - `image = request.files['image']`
  - `image.save('images/uploaded')`

# A bit of practice

- Here we used different port numbers to demonstrate interactions between several machine
- In practice, if the machines are distinct, you use the same port
- Multiple services can run on the same machine
  - One developper writes the maths routes
  - One developper writes the form routes
  - One developper writes the server
- Good practice: don't use http for internal calls (inside the same service)
- But sometimes, it's better (separate teams / rights)

# First example revisited: the backend

In [56]:
%%writefile app_maths.py
from flask import Blueprint
app = Blueprint("maths", __name__)
@app.route("/add/<int:i>/<int:j>")
def add(i,j):
    return [i+j]
@app.route("/sub/<int:i>/<int:j>")
def sub(i,j):
    return [i-j]
@app.route("/mul/<int:i>/<int:j>")
def mul(i,j):
    return [i*j]
@app.route("/div/<int:i>/<int:j>")
def div(i,j):
    return [i//j,i%j]

Overwriting app_maths.py


In [57]:
%%writefile app_strings.py
from flask import Blueprint
app = Blueprint("strings", __name__)
@app.route("/to_upper/<s>")
def to_upper(s):
    return s.upper()

Overwriting app_strings.py


# First example revisited: the frontend

In [58]:
%%writefile app_user.py
from flask import Flask, request
from json import loads
import requests
app = Flask("MyApp")
@app.get("/")
def input_form():
    return """<form action="/" method="post"> i: <input name="i" type="text" /> j: <input name="j" type="text" />
<input type="hidden" name="action" value="add" /> <input value="Add" type="submit" /></form>
<br>
<form action="/" method="post"> Word: <input name="s" type="text" /> <input value="To Uppercase" type="submit" />
 <input type="hidden" name="action" value="up" /> </form>"""
@app.post("/")
def process_form():
    action = request.values['action']
    if action == 'add':
        i = request.values['i']
        j = request.values['j']
        r = requests.get(f"http://127.0.0.1:5000/maths/add/{i}/{j}")
        l = loads(r.text)
        return f"<h1>{i}+{j}={l[0]}</h1>"
    else:
        r = requests.get(f"http://127.0.0.1:5000/strings/to_upper/{request.form['s']}")
        l = r.text
        return f'<h1>"{request.form['s']}".upper()={l}</h1>'    

Overwriting app_user.py


# First example revisited: the server

In [59]:
%%writefile run.py
from app_user import app
from app_maths import app as app_maths
from app_strings import app as app_strings
app.register_blueprint(app_maths, url_prefix="/maths")
app.register_blueprint(app_strings, url_prefix="/strings")
app.run(debug=True)

Overwriting run.py


http://localhost:5000

## Templates

# Templates

In [ ]:
x=2
"x=%s" % x

- Writing nice html requires lot of boring text (tags, etc...)
- With templates, you can prepare skeletons with placeholders to save times
- Many possibilities:
  - Native Python: f-strings, % substitution, string.Template
  - Some frameworks offer advanced templates (e.g. [Jinja](https://jinja.palletsprojects.com/en/latest/templates))
  - Flask is compatible with Jinja renderer

# Basic templates

In [60]:
from jinja2 import Template
tpl = Template('Hello {{name}}!')
print(tpl.render(name='World'))
my_dict={'number': '123', 'street': 'Fake St.', 'city': 'Fakeville', 'key': 'value'}
tpl = Template('The address is at {{number}} {{street}}, {{city}}')
print(tpl.render(**my_dict))

Hello World!
The address is at 123 Fake St., Fakeville


# If condition

In [61]:
tpl = Template('Hello {{name if name != "World" else "Planet"}}!')
print(tpl.render(name="World"))
print(tpl.render(name="Sébastien"))

Hello Planet!
Hello Sébastien!


# Warning: HTML entities

In [62]:
from IPython.display import HTML
tpl = Template('Hello {{name}}!')
HTML(tpl.render(name='<b>World</b>'))

In [63]:
tpl = Template('Hello {{name|e}}!')
HTML(tpl.render(name='<b>World</b>'))

# Template files

In [64]:
%%writefile simple.tpl
<html>
 <head><title>{{title}}</title></head>
 <body>
 <h1>{{title}}</h1>
 {{content}}
 </body>
</html

Overwriting simple.tpl


# Template files

In [65]:
from pathlib import Path
def template(file, **kwargs):
    with open(Path(file), "rt", encoding="utf8") as f:
        tpl = Template(f.read())
    return tpl.render(kwargs)

In [66]:
site = { 'title': 'This is the title', 
'content': 'This is the content'}
s = template('simple.tpl',**site)
print(s)
HTML(s)

<html>
 <head><title>This is the title</title></head>
 <body>
 <h1>This is the title</h1>
 This is the content
 </body>
</html


# Conditions in template files

In [67]:
%%writefile simple_if.tpl
<html>
 <head><title>{{title}}</title></head>
 <body>
 <h1>{{title}}</h1>
{% if bold_content %}
<b>
{% endif %}
 {{content}}
{% if bold_content %}
</b>
{% endif %}
 </body>
</html>

Overwriting simple_if.tpl


In [68]:
site = { 'title': 'This is the title', 
'content': 'This is the content'}
s = template('simple_if.tpl',**site, bold_content=False)
HTML(s)

# Template files loops

In [69]:
%%writefile simple_for.tpl
<html>
 <head><title>{{title}}</title></head>
 <body>
 <h1>{{title}}</h1><ul>
{% for content in contents %}
{% if alternate and loop.index is odd %}
 <li><b>{{content}}</b></li>
{% elif alternate and loop.index is even %}
 <li><em>{{content}}</em></li>
{% else %}
 <li>{{content}}</li>
{% endif %}
{% endfor %}
 </ul></body>
</html>

Overwriting simple_for.tpl


In [70]:
site = { 'title': 'This is the title', 'contents': 
        ['This is the first line of content', 'second line', 'third line']}
s = template('simple_for.tpl', **site, alternate=True)
HTML(s)

# The end!